# Station Usage Simulation Evaluation

Use daily traffic data in 2022 to simulate daily usage.

## Target

1. Analyze the rate in which the stations are built.

2. How many cars are gaing up? At which area?

3. Where should we focus on building new charging station?

## Assumption

- There are about 400 cars using the charging station per day in general within Bristol.
- The average flux of cars in 2022 is 804.6 cars per count point per hour.

# Preperation

In [2]:
# %pwd
%cd '../'
# The directory should be '/electrical_vehicle_project_igp'

/Users/trietdoky/Documents/03 Master/MAIN COURSE/00 IGP/00 Project/source_code/electrical_vehicle_project_igp


In [3]:
import pandas as pd
import numpy as np
import h3
import simpy
from datetime import datetime, timedelta
# import folium     # for jupyter notebooks only
import library.mong_connection as mongo
import library.config as cfg
import data_processing as dpr
import car_simulation as sim
import library.g7_utility as ult
import os
import asyncio

simulation_level = 7
period_start = '2022-01-01 00:00:00'
period_end = '2023-01-01 00:00:00'


## Get Daily Traffic Data

In [4]:
raw_traffic_data = pd.read_csv(os.getcwd()+'/simulation/sys_files/bristol_traffic_density.csv')
# traffic_allocation = dpr.extract_traffic_data()
station_data_simulation = dpr.extract_station_data(simulation_level)
ports_data = dpr.extract_charging_port_data()
raw_station_data, raw_hex_data = dpr.process_traffic_data(simulation_res=simulation_level)
business_ref_table = dpr.generate_business_data(raw_hex_data, raw_station_data)

searching for neighbor of 85195877fffffff
found neighbors of 85195877fffffff with 0
searching for neighbor of 8519583bfffffff
found neighbors of 8519583bfffffff with 0
searching for neighbor of 8519582bfffffff
found neighbors of 8519582bfffffff with 0
searching for neighbor of 85195863fffffff
found neighbors of 85195863fffffff with 1
searching for neighbor of 85195873fffffff
found neighbors of 85195873fffffff with 1
searching for neighbor of 85195867fffffff
found neighbors of 85195867fffffff with 0
searching for neighbor of 8619580cfffffff
found neighbors of 8619580cfffffff with 0
searching for neighbor of 861958397ffffff
found neighbors of 861958397ffffff with 0
searching for neighbor of 86195862fffffff
found neighbors of 86195862fffffff with 1
searching for neighbor of 861958677ffffff
found neighbors of 861958677ffffff with 1
searching for neighbor of 861958647ffffff
found neighbors of 861958647ffffff with 1
searching for neighbor of 861958657ffffff
found neighbors of 861958657ffffff

In [5]:
traffic_analysis_data =  ult.to_camel_case(raw_traffic_data.query('Count_date>=@period_start and Count_date<@period_end').copy().sort_values('Count_date'))
# traffic_analysis_data
date_list = list(np.unique(traffic_analysis_data['CountDate']))
avg_2022_traffic = traffic_analysis_data['AllMotorVehicles'].mean()
# date_list

`traffic_analysis_data` contains all monitored traffic in 2023. The sampling size is 50 days within 2022, usually taken 2 times per week, one of the day is usualy Monday.

## Get Daily Active Station Data

In [6]:
# raw_station_data
raw_simulation_station_data = dpr.extract_station_data(simulation_res=simulation_level)
station_analysis_data = pd.DataFrame()
temp = pd.DataFrame()

for simday in date_list:
    temp = raw_simulation_station_data.query('DateCreated <= @simday and HexLevel == @simulation_level').copy()
    temp['Date'] = simday
    station_analysis_data = pd.concat([station_analysis_data, temp])

# station_analysis_data.to_csv('simulation/output/station_2022.csv')
# station_analysis_data


searching for neighbor of 85195877fffffff
found neighbors of 85195877fffffff with 0
searching for neighbor of 8519583bfffffff
found neighbors of 8519583bfffffff with 0
searching for neighbor of 8519582bfffffff
found neighbors of 8519582bfffffff with 0
searching for neighbor of 85195863fffffff
found neighbors of 85195863fffffff with 1
searching for neighbor of 85195873fffffff
found neighbors of 85195873fffffff with 1
searching for neighbor of 85195867fffffff
found neighbors of 85195867fffffff with 0
searching for neighbor of 8619580cfffffff
found neighbors of 8619580cfffffff with 0
searching for neighbor of 861958397ffffff
found neighbors of 861958397ffffff with 0
searching for neighbor of 86195862fffffff
found neighbors of 86195862fffffff with 1
searching for neighbor of 861958677ffffff
found neighbors of 861958677ffffff with 1
searching for neighbor of 861958647ffffff
found neighbors of 861958647ffffff with 1
searching for neighbor of 861958657ffffff
found neighbors of 861958657ffffff

## Calculate Daily Simulation Parameter

In [17]:
simday = '2022-03-28 00:00:00'
env = simpy.Environment()
tmp_traffic_allocation = dpr.extract_traffic_data_by_date(simday, traffic_count_data=traffic_analysis_data)
tmp_traffic = traffic_analysis_data.query('CountDate == @simday').copy().reset_index()
tmp_station = station_analysis_data.query('Date == @simday').copy().reset_index()
avg_day_traffic = tmp_traffic['AllMotorVehicles'].mean()
daily_user = int(cfg.user_per_day* (ult.sig((avg_day_traffic/avg_2022_traffic)-1)+1))
# daily_user = int(cfg.user_per_day* (avg_day_traffic/avg_2022_traffic))
sim_car_list = sim.generate_car_list(daily_user, tmp_station)
sim_station_list = sim.generate_staion(env, tmp_station)
log_df = pd.DataFrame(columns=['timestamp', 'car_id', 'station_id', 'event_name'])

In [18]:
daily_user

752

In [19]:
sim_result = pd.DataFrame()
# now = datetime.combine(datetime.now().date() + timedelta(days=1), 
#                        datetime.min.time())
# check_list = list(date_list[0:2])
for simday in date_list:
# for simday in check_list:
    sim_date = datetime.strptime(simday, '%Y-%m-%d %H:%M:%S')
    env = simpy.Environment()
    tmp_traffic_allocation = dpr.extract_traffic_data_by_date(simday, traffic_count_data=traffic_analysis_data)
    tmp_traffic = traffic_analysis_data.query('CountDate == @simday').copy().reset_index()
    tmp_station = station_analysis_data.query('Date == @simday').copy().reset_index()
    avg_day_traffic = tmp_traffic['AllMotorVehicles'].mean()
    daily_user = int(cfg.user_per_day*(ult.sig((avg_day_traffic/avg_2022_traffic)-1)+1))
    # daily_user = int(cfg.user_per_day* (avg_day_traffic/avg_2022_traffic))
    sim_car_list = sim.generate_car_list(daily_user, tmp_station)
    sim_station_list = sim.generate_staion(env, tmp_station)
    log_df = pd.DataFrame(columns=['timestamp', 'car_id', 'station_id', 'event_name', 'elapsed_time'])
    for id, ev in sim_car_list.iterrows():
        chosen_station = ev['ChargeDeviceId']
        station_info = dict(sim_station_list[sim_station_list['ChargeDeviceId'] == chosen_station].head(1))
        charge_time = ev['ChargeTime']
        car_id = ev['CarId']
        resource = station_info['Station'].values[0]
        env.process(sim.car(env,car_id,chosen_station,resource,charge_time,sim_date,log_df,traffic_allocation=tmp_traffic_allocation))
    env.run()
    log_df['date'] = simday
    sim_result = pd.concat([sim_result, log_df])
sim_result.to_csv('simulation/output/simulation_all_2022.csv', index=False)

# Runtime: 30s
# 45k rows



## Analysis

There are 4 possible status for cars interacting with the stations:

- 'arrive - charge': Arrive and charge right away 

- 'arrive - queue': Arrive but have to take a queue

- 'charge - finish': Finish charging and leave

- 'queue - charge': Queue then find a spot and start charging

- 'queue - give up': Queue too long and have to give up

Decided to use R for simpler visualizations.

In [28]:
sim_result = pd.read_csv('simulation/output/simulation_all_2022.csv')

In [33]:
sim_result.head(100)

,timestamp,car_id,station_id,event_name,date
0,2022-03-21 02:18:40,bes0397,63ba327b26344b5f629861defddc2099,arrive - charge,2022-03-21 00:00:00
1,2022-03-21 02:41:14,bes0397,63ba327b26344b5f629861defddc2099,charge - finish,2022-03-21 00:00:00
2,2022-03-21 02:56:38,bes0272,a39256a0972490978df0b4d85c5736be,arrive - charge,2022-03-21 00:00:00
3,2022-03-21 03:30:53,bes0088,6cc14668d720ff3088c3e2325745080c,arrive - charge,2022-03-21 00:00:00
4,2022-03-21 03:35:07,bes0123,90d60beb748605ceebd6f17551dd5a1d,arrive - charge,2022-03-21 00:00:00
...,...,...,...,...,...
95,2022-03-21 08:09:42,bes0131,e757852199c96e39152d28c1efea00c0,arrive - charge,2022-03-21 00:00:00
96,2022-03-21 08:11:06,bes0361,1b7f860f0874f582a9118ec56a879e56,charge - finish,2022-03-21 00:00:00
97,2022-03-21 08:12:15,bes0329,90d60beb748605ceebd6f17551dd5a1d,arrive - charge,2022-03-21 00:00:00
98,2022-03-21 08:13:11,bes0336,d30cca09053fc7dffbf5576cde094c45,arrive - charge,2022-03-21 00:00:00


In [30]:
np.unique(sim_result['event_name'])

array(['arrive - charge', 'arrive - queue', 'charge - finish',
       'queue - charge', 'queue - give up'], dtype=object)

In [32]:
count_table = (sim_result.groupby(['date'], as_index=False)
               .agg(
                   count_car_demand = ('car_id', 'nunique'),
                   count_statio_used = ('station_id', 'nunique'),
                   count_queue_car = ('event')
                   ))
count_table

,date,count_car,count_station
0,2022-03-21 00:00:00,503,109
1,2022-03-25 00:00:00,196,87
2,2022-03-28 00:00:00,1232,115
3,2022-03-29 00:00:00,1237,115
4,2022-04-08 00:00:00,1651,113
5,2022-04-27 00:00:00,215,93
6,2022-04-29 00:00:00,659,111
7,2022-05-05 00:00:00,741,110
8,2022-05-06 00:00:00,2031,114
9,2022-05-09 00:00:00,1392,115
